In [1]:
!pip install numpy pandas scikit-learn tensorflow keras matplotlib


In [2]:
!wget https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.csv
!wget https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.csv


--2025-12-04 07:22:15--  https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14612857 (14M) [text/plain]
Saving to: ‘KDDTrain+.csv’

KDDTrain+.csv       100%[===================>]  13.94M  --.-KB/s    in 0.05s   

2025-12-04 07:22:16 (285 MB/s) - ‘KDDTrain+.csv’ saved [14612857/14612857]

--2025-12-04 07:22:16--  https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2639177 (2.5M) [text/plain]
Saving

In [3]:
import pandas as pd

cols = [
"duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment",
"urgent","hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted",
"num_root","num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login",
"is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate",
"srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count",
"dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
"dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
"dst_host_rerror_rate","dst_host_srv_rerror_rate","label"
]

train = pd.read_csv("KDDTrain+.csv", names=cols)
test = pd.read_csv("KDDTest+.csv", names=cols)


In [6]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

cat_cols = ["protocol_type","service","flag"]
for col in cat_cols:
    le = LabelEncoder()
    # Fit LabelEncoder on the combined unique values from both train and test sets, ensuring string type
    le.fit(pd.concat([train[col], test[col]], axis=0).astype(str).unique())
    # Transform the columns, ensuring string type for consistent input with the fitted encoder
    train[col] = le.transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))

In [8]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

scaler = MinMaxScaler()

# Drop the 'label' column and select only numeric columns for scaling.
# This ensures that any remaining categorical columns that were not
# successfully converted to numbers are excluded, preventing the ValueError.
X_train_processed = train.drop("label", axis=1).select_dtypes(include=['number'])
X_test_processed = test.drop("label", axis=1).select_dtypes(include=['number'])

X_train = scaler.fit_transform(X_train_processed)
X_test  = scaler.transform(X_test_processed)

In [9]:
y_train = (train['label'] != 'normal').astype(int)
y_test = (test['label'] != 'normal').astype(int)


In [10]:
import numpy as np

X_train_lstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm  = np.reshape(X_test,  (X_test.shape[0], 1, X_test.shape[1]))


In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(1, 39))) # Changed input_shape from 41 to 39
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 1, 64)          │        26,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,162 (156.88 KB)

 Trainable params: 40,162 (156.88 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(
    X_train_lstm, y_train,
    epochs=10,
    batch_size=128,
    validation_split=0.1
)


Epoch 1/10
886/886 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9972 - loss: 0.0977 - val_accuracy: 1.0000 - val_loss: 1.6332e-05
Epoch 2/10
886/886 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 1.0000 - loss: 1.5407e-05 - val_accuracy: 1.0000 - val_loss: 3.8462e-06
Epoch 3/10
886/886 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 1.0000 - loss: 4.4939e-06 - val_accuracy: 1.0000 - val_loss: 1.4979e-06
Epoch 4/10
886/886 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 1.0000 - loss: 1.9223e-06 - val_accuracy: 1.0000 - val_loss: 7.1282e-07
Epoch 5/10
886/886 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 1.0000 - loss: 9.3165e-07 - val_accuracy: 1.0000 - val_loss: 3.7261e-07
Epoch 6/10
886/886 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 1.0000 - loss: 5.3491e-07 - val_accuracy: 1.0000 - val_loss: 1.9863e-07
Epoch 7/10
886/886 ━━━━━━━━━━━━━━━━━━━━ 14s 14ms/step - accuracy: 1.0000 - loss: 3.1102e-07 - val_accuracy: 1.0000 - val_loss: 1.0611e-07
Epoch 8/10
886/886 ━━━━━━━━━━━━━━━━━━━━ 12s 13

In [16]:
from keras.models import Model

feature_extractor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

X_train_features = feature_extractor.predict(X_train_lstm)
X_test_features  = feature_extractor.predict(X_test_lstm)

3937/3937 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step
705/705 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [17]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_features, y_train)


KNeighborsClassifier()

In [18]:
from sklearn.metrics import accuracy_score, classification_report

pred = knn.predict(X_test_features)
print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     22543

    accuracy                           1.00     22543
   macro avg       1.00      1.00      1.00     22543
weighted avg       1.00      1.00      1.00     22543

